In [1]:
import twitter
import os
from config import *
import time

# initialize api instance
twitter_api = twitter.Api(consumer_key=consumer_key,
                         consumer_secret=consumer_secret,
                         access_token_key=access_token_key,
                         access_token_secret=access_token_secret,
                        cache=None,tweet_mode='compat')

#print(consumer_key,consumer_secret,access_token_key,access_token_secret)

# test authentication
print(twitter_api.VerifyCredentials())
# print(twitter_api.GetStatus(126402758403305000))

{"created_at": "Wed Jul 20 02:34:27 +0000 2016", "description": "I am Al and I am passionate about technology.", "followers_count": 5, "friends_count": 52, "id": 755591669194514432, "id_str": "755591669194514432", "location": "Tustin, CA", "name": "Al", "profile_background_color": "000000", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme1/bg.png", "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme1/bg.png", "profile_banner_url": "https://pbs.twimg.com/profile_banners/755591669194514432/1469041661", "profile_image_url": "http://pbs.twimg.com/profile_images/755843006062694401/2JikSpJq_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/755843006062694401/2JikSpJq_normal.jpg", "profile_link_color": "1B95E0", "profile_sidebar_border_color": "000000", "profile_sidebar_fill_color": "000000", "profile_text_color": "000000", "screen_name": "alknowstech", "status": {"created_at": "Mon Jul 31 21:38:46 +0000 2017", 

In [2]:
def buildTestSet(search_keyword):
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count = 100,
                                               lang='en',result_type='recent')
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [{"text":status.text, "label":None, "user":status.user.screen_name, "time":status.created_at} for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

In [3]:
# test search function
search_term = input("Enter a search keyword:")
testDataSet = buildTestSet(search_term)

print(testDataSet[0:10])

Enter a search keyword:amazon
Fetched 100 tweets for the term amazon
[{'text': 'RT @SeanSupplee: 💵 $250 Amazon Gift Card #Giveaway \n\nHow To Enter:\n🔗 https://t.co/BOARjcVJQb\n\nBonus:\n💞 Retweet &amp; Like\n🔔 Follow @SeanSupplee…', 'label': None, 'user': 'danob52', 'time': 'Tue Feb 18 02:48:57 +0000 2020'}, {'text': "RT @HLfavorite: ★★★★★ –Alexis-\n'A well-told and fast-paced novel, a story of life, love, passion, courage, dedication, struggle, strength,…", 'label': None, 'user': 'petercbradbury', 'time': 'Tue Feb 18 02:48:56 +0000 2020'}, {'text': "RT @sandysperling3: HERE'S AN IDEA FOR WOMEN RELUCTANT TO BUY THIS BOOK!!\nhttps://t.co/mnpMbDuwQw\n#IAN1   https://t.co/usfeEOAAhs https://t…", 'label': None, 'user': 'WBooneHedgepeth', 'time': 'Tue Feb 18 02:48:56 +0000 2020'}, {'text': 'RT @TandonRaveena: Just watched The movie “ Batla House”  on Amazon Prime. Loved it .  A Must watch.', 'label': None, 'user': 'rakesh26329280', 'time': 'Tue Feb 18 02:48:55 +0000 2020'}, {'text': 'RT @n

# Use to test twitter api

In [ ]:
tweets_fetched = twitter_api.GetSearch('disney', count = 100, lang='en')
[{"text":status.text, "label":None, "user":status.user.screen_name} for status in tweets_fetched]

# Start preprocessing of training data

In [4]:
import pandas as pd
import numpy as np

traindata140 = os.path.join(".","trainingandtestdata_sentiment140","training.csv")
testdata140 = os.path.join(".","trainingandtestdata_sentiment140","test.csv")

In [5]:
#read the csv file command
train_df = pd.read_csv(traindata140, header=None, usecols=[0,5], names=['polarity of the tweet','text'], 
                       encoding="ISO-8859-1")
train_df.head()
# (0 = negative, 2 = neutral, 4 = positive)

,polarity of the tweet,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


# Test Df Validation

In [6]:
test_df = pd.read_csv(testdata140, header=None, usecols=[0,5],
                      names=['polarity of the tweet', 'text'],encoding="ISO-8859-1")
conditions_test = [
    (test_df['polarity of the tweet'] == 0),
    (test_df['polarity of the tweet'] == 2),
    (test_df['polarity of the tweet'] == 4)]
choices_test = ['negative', 'neutral', 'positive']
test_df['label'] = np.select(conditions_test, choices_test)
test_df.head()

,polarity of the tweet,text,label
0,4,@stellargirl I loooooooovvvvvveee my Kindle2. ...,positive
1,4,Reading my kindle2... Love it... Lee childs i...,positive
2,4,"Ok, first assesment of the #kindle2 ...it fuck...",positive
3,4,@kenburbary You'll love your Kindle2. I've had...,positive
4,4,@mikefish Fair enough. But i have the Kindle2...,positive


In [7]:
test_ls = test_df.to_dict('records')

In [8]:
train_df_negative = train_df.loc[train_df['polarity of the tweet']==0]

In [9]:
train_df_positive = train_df.loc[train_df['polarity of the tweet']==4]

In [10]:
frames = [train_df_negative.iloc[0:15000,:],train_df_positive.iloc[0:15000,:]]

In [11]:
result = pd.concat(frames)

In [12]:
conditions = [
    (result['polarity of the tweet'] == 0),
    (result['polarity of the tweet'] == 2),
    (result['polarity of the tweet'] == 4)]
choices = ['negative', 'neutral', 'positive']
result['label'] = np.select(conditions, choices)
result.head()

,polarity of the tweet,text,label
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",negative
1,0,is upset that he can't update his Facebook by ...,negative
2,0,@Kenichan I dived many times for the ball. Man...,negative
3,0,my whole body feels itchy and like its on fire,negative
4,0,"@nationwideclass no, it's not behaving at all....",negative


In [13]:
train_ls = result.to_dict('records')

In [14]:
import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
import nltk
# nltk.download('punkt')

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]
    
tweetProcessor = PreProcessTweets()


In [15]:
# Process Training Set
preprocessedTrainingSet = tweetProcessor.processTweets(train_ls)

In [16]:
# Process Test Set
preprocessedTestSet = tweetProcessor.processTweets(test_ls)

In [17]:
import nltk 

def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [18]:
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

In [19]:
word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)

In [20]:
from nltk.classify import SklearnClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC

SKLearnClassifier = SklearnClassifier(BernoulliNB()).train(trainingFeatures)
# SKLearnClassifier = SklearnClassifier(SVC(), sparse=False).train(trainingFeatures)


In [21]:
SKLearnResultLabels = [SKLearnClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]


In [22]:
# Save Pickle
import pickle
save_classifier = open("sklearn_nbayes_train_30k.pickle","wb")
pickle.dump(SKLearnClassifier, save_classifier)
save_classifier.close()

In [ ]:
# Load Pickle
import pickle
classifier_f = open("sklearn_nbayes_train_25k.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()

In [ ]:
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# get the majority vote
if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
    print("Overall Positive Sentiment")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
else: 
    print("Overall Negative Sentiment")
    print("Negative Sentiment Percentage = " + str(100*NBResultLabels.count('negative')/len(NBResultLabels)) + "%")

In [ ]:
NBResultLabelsPickle = [classifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# get the majority vote
if NBResultLabelsPickle.count('positive') > NBResultLabelsPickle.count('negative'):
    print("Overall Positive Sentiment")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabelsPickle.count('positive')/len(NBResultLabelsPickle)) + "%")
else: 
    print("Overall Negative Sentiment")
    print("Negative Sentiment Percentage = " + str(100*NBResultLabelsPickle.count('negative')/len(NBResultLabelsPickle)) + "%")

In [23]:
# had to add space before tweet text to eliminate csv issue with tweets that begins with @
# list comprehension
test_list_text = [' '+test['text'] for test in test_ls]
# convert twitter time output to real time output
# test_list_time = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(test['time'],'%a %b %d %H:%M:%S +0000 %Y')) 
# for test in testDataSet]

In [24]:
output_list = pd.DataFrame(
    {'Tweet': test_list_text,
#      'Time': test_list_time,
     'Result': SKLearnResultLabels
    })

In [25]:
output_list

,Tweet,Result
0,@stellargirl I loooooooovvvvvveee my Kindle2....,positive
1,Reading my kindle2... Love it... Lee childs ...,positive
2,"Ok, first assesment of the #kindle2 ...it fuc...",positive
3,@kenburbary You'll love your Kindle2. I've ha...,positive
4,@mikefish Fair enough. But i have the Kindle...,positive
5,@richardebaker no. it is too big. I'm quite h...,positive
6,Fuck this economy. I hate aig and their non l...,negative
7,Jquery is my new best friend.,positive
8,Loves twitter,positive
9,how can you not love Obama? he makes jokes ab...,positive


In [26]:
output_list.to_csv('./outputSKLearnNBayes30k.csv',header=True,
                              index=False,encoding="utf-8")